In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [11]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

# hier ein Test von Barnaby, wie man die Namen alleine mit "Split" separieren kann

- Vielleicht zuerst noch mit Pandas nur die Vorstösse raussuchen. Ist ja das, was mich interessiert.
- Rausfinden, wie ich die Namen nun rausziehe und in ein aufklappbares Dataframe stelle


In [12]:

def splitting(elem):
    elem = elem.split('betreffend')[0].strip()
    if 'von' in elem:
        elem = elem.split('von')[1].strip()
        elem = elem.replace(' und', ',')
    elif 'der' in elem:
        elem = elem.split('der')[1].strip()    
    return elem
df_vs["Geschäft"]=df_vs['Geschäft'].apply(splitting) #.apply ist um eine Funktion auf eine ganze Reihe Pandas einzusetze

In [13]:
df_fgraph=df_vs[["Geschäft","VorlageNR"]]# Hier mache ich ein DF mit welchem ich mir erhoffe einen Graphen zu machen.

In [14]:
df_fgraph.to_csv("KRFrames/Geschaeft_fuer_graph")

# Nun rausfinden, wie ich ein DataFrame machen kann, das man aufklappt

In [4]:
#das funktionerit also nicht
def splitkomma(elem):
    elem=elem.split(", ")
    return elem


In [5]:
df_vs["Geschäft"]=df_vs["Geschäft"].apply(splitkomma) 

In [9]:
df_vs.to_csv("KRFrames/Namen_in_Liste")

In [1]:
# Hier die Lösung, wie ich für jeden Namen ein einzelnes Geschäft erhalte

In [10]:
df_ex=df_vs.explode('Geschäft').reset_index(drop=True)

In [11]:
df_ex.rename(columns={'Geschäft':'Namenganz'}, inplace=True)

In [2]:
df_ex.to_csv("KRFrames/df_ex")

NameError: name 'df_ex' is not defined

In [3]:
df_ex=pd.read_csv("KRFrames/df_ex")

In [4]:
df_ex

,Unnamed: 0,Unnamed: 0.1,level_0,index,VorlageNR,url,Namenganz,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,0,0,3031,/gast/geschaefte/2018,SVP-Fraktion,Einreichung,Direktion für Bildung und Kultur,Interpellation,hängig
1,1,1,1,1,3030,/gast/geschaefte/2017,SVP-Fraktion,Einreichung,NaN,Motion,hängig
2,2,2,2,2,3029,/gast/geschaefte/2016,Fraktion Alternative - die Grünen,Einreichung,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3,5,3026,/gast/geschaefte/2013,Markus Spörri,Einreichung,Baudirektion,Interpellation,hängig
4,4,3,3,5,3026,/gast/geschaefte/2013,Thomas Gander,Einreichung,Baudirektion,Interpellation,hängig
...,...,...,...,...,...,...,...,...,...,...,...
1955,1955,1222,1222,1957,762,/gast/geschaefte/1331,erweiterten Justizprüfungskommission,Datenmigration,NaN,Motion,abgeschlossen
1956,1956,1223,1223,1963,666,/gast/geschaefte/1325,Kommission Teilrevision Personalgesetz,Datenmigration,NaN,Motion,abgeschlossen
1957,1957,1224,1224,1979,304,/gast/geschaefte/1309,Manuela Weichelt,Datenmigration,NaN,Motion,abgeschlossen
1958,1958,1225,1225,1980,282,/gast/geschaefte/1712,Christoph Hohler,Frist erstreckt,Baudirektion,Motion,hängig


In [13]:
# hier schaue ich, wie gross das Dataframe ohne die Kommissionen und so ist.
df_ex_cl=df_ex[df_ex["Namenganz"].str.contains("Kommission|kommission")==False]

In [14]:
df_ex_cl=df_ex_cl[["VorlageNR", "Namenganz", ]]

In [15]:
df_exneu=df_ex_cl.head(13)# hier die neuen Namen ab Dezember in ein Dataframe

In [ ]:
# ICH bereinige von Hand im Excel
df_ex_cl.to_excel("KRFrames/df_ex_cl.xlsx")

In [ ]:
df_ex_cl[df_ex_cl["Namenganz"].str.contains("zur")]
#oook, die Alternativen wurden unteschiedliche benamselt. Das muss ich genauer anschauen.
# Auch andere Parteien werden zum Teil doppelt genannt. 

# Bereinigen der Parteinamen
Diese werden in mindestens drei unterschiedlichen Arten genannt

### Ich habe mich fürs Bereinigen im Excel entschieden, da Daten noch übersichtlich sind

In [6]:
# Hier nun das im Excel bereinigte Sheet
df_ex_1=pd.read_excel("KRFrames/df_ex_cl_geputzt.xlsx")

In [7]:
df_ex_1

,Unnamed: 0,VorlageNR,Namenganz
0,0,3020,Luzian Franzini
1,1,3020,Esther Haas
2,2,3019,Luzian Franzini
3,3,3019,Rita Hofer
4,4,3019,Tabea Zimmermann Gibson
...,...,...,...
1910,1937,836,Käty Hofer Buser
1911,1940,810,Marcel Meyer
1912,1941,801,Hans Abicht
1913,1944,304,Manuela Weichelt


In [8]:
df_ex1=df_ex_1[["VorlageNR", "Namenganz"]]

In [16]:
df_exneu["Namenganz"]=df_exneu["Namenganz"].str.replace("Fraktion Alternative - die Grünen","Alternative")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_exneu

,VorlageNR,Namenganz
0,3031,SVP-Fraktion
1,3030,SVP-Fraktion
2,3029,Alternative
3,3026,Markus Spörri
4,3026,Thomas Gander
5,3024,Claus Soltermann
6,3024,Heinz Achermann
7,3023,SP-Fraktion
8,3021,Philip C. Brunner
9,3021,Barbara Gysel


In [18]:
df_ex1

,VorlageNR,Namenganz
0,3020,Luzian Franzini
1,3020,Esther Haas
2,3019,Luzian Franzini
3,3019,Rita Hofer
4,3019,Tabea Zimmermann Gibson
...,...,...
1910,836,Käty Hofer Buser
1911,810,Marcel Meyer
1912,801,Hans Abicht
1913,304,Manuela Weichelt


In [19]:
#Zusammenführen mit den neuen DAten
df_ex_ber=pd.concat([df_ex1,df_exneu])

In [20]:
df_ex_ber.sort_values("VorlageNR")

,VorlageNR,Namenganz
1914,282,Christoph Hohler
1913,304,Manuela Weichelt
1912,801,Hans Abicht
1911,810,Marcel Meyer
1910,836,Käty Hofer Buser
...,...,...
3,3026,Markus Spörri
4,3026,Thomas Gander
2,3029,Alternative
1,3030,SVP-Fraktion


### Problem erkannt: 
offenbar werden gekoppelte Namen in den Geschäften nur mit den ersten aufgeführt.
Ich ergänze das Excel mit den Doppelnamen in unterschiedlichen Schreibweisen, bis alle abgedeckt sind.

In [21]:
df_ex_ber

,VorlageNR,Namenganz
0,3020,Luzian Franzini
1,3020,Esther Haas
2,3019,Luzian Franzini
3,3019,Rita Hofer
4,3019,Tabea Zimmermann Gibson
...,...,...
8,3021,Philip C. Brunner
9,3021,Barbara Gysel
10,3021,Anna Spescha
11,3021,Andreas Lustenberger


In [22]:
df_ex_ber.shape

(1928, 2)

In [23]:
# Nun das Frame mit den Namen einlesen
df_np=pd.read_excel('KRFrames/Alle_Namen_mit_Allianz_und_ohne_ergänzt.xlsx')

In [24]:
#von der Bearbeitung hat es nun noch einen anderen Index. stört uns nicht - Wir entfernen "unnamed:0"
del df_np["Unnamed: 0"]

In [25]:
df_np.shape #voila

(415, 3)

In [26]:
df_np

,Namenganz,P,Unnamed: 0.1
0,Judith Wild,FDP,NaN
1,Madeleine Landolt,SGA,NaN
2,Peter Bossard,FDP,NaN
3,Susi Frei,FDP,NaN
4,Alois Henggeler,CVP,NaN
...,...,...,...
410,Othmar Birri,SP,NaN
411,Gabriela Peita,SP,NaN
412,Andrea Erni,SP,NaN
413,Rolf Schweiger,FDP,NaN


In [27]:
#Nun die beiden zusammenfügen - an den Namen
df_alles=pd.merge(df_np, df_ex_ber, how="inner",on= "Namenganz", )

In [28]:
df_alles.shape

(1919, 4)

In [29]:
df_alles.sort_values("P").tail(35)

,Namenganz,P,Unnamed: 0.1,VorlageNR
1092,Jürg Messmer,SVP,NaN,2307
495,Thomas Villiger,SVP,NaN,2525
517,Stephan Schleiss,SVP,NaN,1537
496,Thomas Villiger,SVP,NaN,2220
498,Thomas Villiger,SVP,NaN,1743
515,Stephan Schleiss,SVP,NaN,1652
514,Stephan Schleiss,SVP,NaN,1711
513,Stephan Schleiss,SVP,NaN,1736
512,Stephan Schleiss,SVP,NaN,1813
511,Stephan Schleiss,SVP,NaN,1832


In [30]:
df_alles.sort_values("VorlageNR")

,Namenganz,P,Unnamed: 0.1,VorlageNR
24,Christoph Hohler,SP,NaN,282
25,Manuela Weichelt,ALG,NaN,304
23,Hans Abicht,FDP,NaN,801
14,Marcel Meyer,CVP,NaN,810
110,Käty Hofer Buser,SP,NaN,836
...,...,...,...,...
1352,Markus Spörri,FDP,NaN,3026
1217,Thomas Gander,FDP,NaN,3026
1918,Alternative,ALG,NaN,3029
1445,SVP-Fraktion,SVP,NaN,3030


In [32]:
df_alles[df_alles["Namenganz"].str.contains("Jolanda")]

,Namenganz,P,Unnamed: 0.1,VorlageNR
1809,Jolanda Spiess-Hegglin,Piraten,302.0,2707
1810,Jolanda Spiess-Hegglin,Piraten,302.0,2706
1811,Jolanda Spiess-Hegglin,Piraten,302.0,2705
1812,Jolanda Spiess-Hegglin,Piraten,302.0,2703
1813,Jolanda Spiess-Hegglin,Piraten,302.0,2702
1814,Jolanda Spiess-Hegglin,Piraten,302.0,2701
1815,Jolanda Spiess-Hegglin,Piraten,302.0,2700
1816,Jolanda Spiess-Hegglin,Piraten,302.0,2699
1817,Jolanda Spiess-Hegglin,Piraten,302.0,2698
1818,Jolanda Spiess-Hegglin,Piraten,302.0,2696


In [ ]:
# So, es sind noch einige wenige, die offenbar fehlerhaft sind
# Da es nur noch einzelne Positionen betrifft, finde ich es so jetzt ok.
                            

In [33]:

del df_alles["Unnamed: 0.1"]
# nun das df_alles noch abspeichern für den Endcode
df_alles.to_csv("KRFrames/Frame_mit_Namen_und_Parteien")

In [34]:
df_alles.dtypes

Namenganz    object
P            object
VorlageNR     int64
dtype: object

# Bereinigung, Parteien und Doppelnamen
offenbar werden bei mehrern Parteien diese nicht richtig getrennt. ich werde diese nun:
- dataframes mit str.contains machen und die Parteien rausfiltern und sie richtig benennen
- dazu sage ich allen jeweiligen Spalten, sie sollen nun z.b. SVP-Fraktion heissen.
- Fäür jede Patei inkl Vorlagenummer ein neues Dataframe abspeichern
- alle an die Namensliste hänen
- Dublikate bei Namen und Vorlagenummern löschen

In [35]:
df_alles1=pd.read_csv("KRFrames/Frame_mit_Namen_und_Parteien")

In [36]:
df_alles1

,Unnamed: 0,Namenganz,P,VorlageNR
0,0,Peter Rust,CVP,1432
1,1,Peter Rust,CVP,1388
2,2,Peter Rust,CVP,1383
3,3,Peter Rust,CVP,1324
4,4,Peter Rust,CVP,991
...,...,...,...,...
1914,1914,Othmar Birri,SP,1186
1915,1915,Gabriela Peita,SP,2266
1916,1916,Andrea Erni,SP,1229
1917,1917,Rolf Schweiger,FDP,1426


In [37]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [38]:
df_vs1=df_vs[["VorlageNR", "Geschäft"]]

In [39]:
#die Dataframes
df_alg=df_vs1[df_vs1["Geschäft"].str.contains("Alternative")]
df_sp=df_vs1[df_vs1["Geschäft"].str.contains("SP-Fraktion")]
df_cvp=df_vs1[df_vs1["Geschäft"].str.contains("CVP-Fraktion")]
df_fdp=df_vs1[df_vs1["Geschäft"].str.contains("FDP-Fraktion")]
df_glp=df_vs1[df_vs1["Geschäft"].str.contains("GLP-Fraktion")]
df_svp=df_vs1[df_vs1["Geschäft"].str.contains("SVP-Fraktion")]

In [40]:
 df_alg.shape,df_sp.shape,df_cvp.shape, df_fdp.shape, df_glp.shape, df_svp.shape

((106, 2), (97, 2), (82, 2), (35, 2), (0, 2), (76, 2))

In [41]:
df_vs[df_vs["Geschäft"].str.contains("das alternative")]
#"Fraktion Alternative", "Alternativen"

,Unnamed: 0,level_0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status


In [42]:
df_sp 

,VorlageNR,Geschäft
5,3023,Interpellation der SP-Fraktion betreffend «Dre...
12,3013,Interpellation der SP-Fraktion betreffend Wohn...
16,3007,Interpellation der SP-Fraktion betreffend Pest...
27,2992,Postulat der SP-Fraktion und der Fraktion Alte...
31,2983,Kleine Anfrage der SP-Fraktion betreffend Kamp...
...,...,...
1087,1253,Interpellation der SP-Fraktion betreffend Schw...
1103,1211,Motion der SP-Fraktion betreffend Einreichung ...
1112,1188,Motion der SP-Fraktion betreffend regelmässige...
1113,1187,Interpellation der Alternativen Fraktion und d...


In [43]:
#hier eine Funktion, um bei allen Namen die jeweilige Partei einzufügen.
def pareiersetz(satz):
    satz=partei
    return satz        

In [44]:
partei="SVP-Fraktion"
df_svp["Geschäft"]=df_svp["Geschäft"].apply(pareiersetz)
partei="FDP-Fraktion"
df_fdp["Geschäft"]=df_fdp["Geschäft"].apply(pareiersetz)
partei="CVP-Fraktion"
df_cvp["Geschäft"]=df_cvp["Geschäft"].apply(pareiersetz)
partei="GLP-Fraktion"
df_glp["Geschäft"]=df_glp["Geschäft"].apply(pareiersetz)
partei="SP-Fraktion"
df_sp["Geschäft"]=df_sp["Geschäft"].apply(pareiersetz)
partei="Alternative"
df_alg["Geschäft"]=df_alg["Geschäft"].apply(pareiersetz)

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [45]:
df_cvp

,VorlageNR,Geschäft
18,3004,CVP-Fraktion
24,2995,CVP-Fraktion
43,2966,CVP-Fraktion
44,2962,CVP-Fraktion
50,2952,CVP-Fraktion
...,...,...
1145,1133,CVP-Fraktion
1181,1054,CVP-Fraktion
1189,1041,CVP-Fraktion
1199,1008,CVP-Fraktion


In [46]:
#zuerst die Parteiframes zusammenfügen
frames=[df_svp, df_fdp, df_glp, df_cvp, df_sp, df_alg]
df_alpart = pd.concat(frames,sort=True)


In [47]:
df_alpart.shape

(396, 2)

In [48]:
#nun den Namen dem Frame Namenganz anpassen
df_alpart.rename(columns={"Geschäft":"Namenganz"}, inplace=True)

In [49]:
df_alpart.groupby("Namenganz").count().sort_values("VorlageNR")

,VorlageNR
Namenganz,
FDP-Fraktion,35
SVP-Fraktion,76
CVP-Fraktion,82
SP-Fraktion,97
Alternative,106


In [50]:
# nun muss ich das dataframe noch mit jenem der Parteien zusammenfügen
df_alpart1=pd.merge(df_alpart, df_np, how="inner",on= "Namenganz")

In [51]:
df_alpart1.groupby("Namenganz").count().sort_values("P")

,VorlageNR,P,Unnamed: 0.1
Namenganz,,,
FDP-Fraktion,35,35,0
SVP-Fraktion,76,76,0
CVP-Fraktion,82,82,0
SP-Fraktion,97,97,0
Alternative,106,106,0


In [52]:
df_alles1.shape, df_alpart1.shape

((1919, 4), (396, 4))

In [53]:
#nun mit df_alles zusammenfügen
newframes=[df_alles1, df_alpart1]
df_alles2=pd.concat(newframes, sort=True)
df_alles2.shape

(2315, 5)

In [54]:
df_alles2.sort_values("P").tail(36)

,Namenganz,P,Unnamed: 0,Unnamed: 0.1,VorlageNR
1437,SVP-Fraktion,SVP,1437.0,NaN,1323
1418,SVP-Fraktion,SVP,1418.0,NaN,1845
1417,SVP-Fraktion,SVP,1417.0,NaN,1858
1422,SVP-Fraktion,SVP,1422.0,NaN,1744
1415,SVP-Fraktion,SVP,1415.0,NaN,1912
1396,SVP-Fraktion,SVP,1396.0,NaN,2500
1397,SVP-Fraktion,SVP,1397.0,NaN,2479
1398,SVP-Fraktion,SVP,1398.0,NaN,2376
1399,SVP-Fraktion,SVP,1399.0,NaN,2370
1400,SVP-Fraktion,SVP,1400.0,NaN,2344


In [55]:
df_alles2.groupby("Namenganz").count().sort_values("P")

,P,Unnamed: 0,Unnamed: 0.1,VorlageNR
Namenganz,,,,
Jost Arnold,1,1,0,1
Rolf Schweiger,1,1,0,1
Gerhard Pfister,1,1,0,1
Karin Julia Stadlin,1,1,0,1
Gabriela Peita,1,1,0,1
...,...,...,...,...
Alternativen Fraktion,97,97,0,97
Alternative,107,1,0,107
SVP-Fraktion,149,73,0,149


In [56]:
# habe gemerkt, dass ich da ja noch die Alternativen beides Mal gleich nennen muss, sonst werden sie nicht rausgeschmissen
df_alles2["Namenganz"]=df_alles2["Namenganz"].str.replace("Alternativen Fraktion", "Alternative")

In [57]:
df_alles2.groupby("VorlageNR").count().sort_values("Namenganz", ascending=False)

,Namenganz,P,Unnamed: 0,Unnamed: 0.1
VorlageNR,,,,
2728,14,14,14,0
2049,12,12,12,0
2586,10,10,7,0
3002,9,9,9,0
2591,9,9,9,0
...,...,...,...,...
2079,1,1,1,0
304,1,1,1,0
2096,1,1,1,0


# Erkenntnis: 
ich muss die Namen bereinigen für das Ranking. Sonst werden diese nicht alle als solche erkannt.Und fallen hier durch. 
## und Aaahhh, beim ersten Durchgang hatte ich diese im Excel geputzt. 
### Muss also so was nur noch für die mit Doppelnamen machen

In [58]:
df_alles.shape

(1919, 3)

In [59]:
df_allesgut=df_alles2.drop_duplicates(subset=['VorlageNR', 'Namenganz'])

In [60]:
df_allesgut

,Namenganz,P,Unnamed: 0,Unnamed: 0.1,VorlageNR
0,Peter Rust,CVP,0.0,NaN,1432
1,Peter Rust,CVP,1.0,NaN,1388
2,Peter Rust,CVP,2.0,NaN,1383
3,Peter Rust,CVP,3.0,NaN,1324
4,Peter Rust,CVP,4.0,NaN,991
...,...,...,...,...,...
339,Alternative,ALG,NaN,NaN,1932
358,Alternative,ALG,NaN,NaN,1694
359,Alternative,ALG,NaN,NaN,1679
363,Alternative,ALG,NaN,NaN,1608


In [61]:
df_allesgut.groupby("Namenganz").count().sort_values("P", ascending=False).head(15)

,P,Unnamed: 0,Unnamed: 0.1,VorlageNR
Namenganz,,,,
Alternative,106,98,0,106
SP-Fraktion,101,87,0,101
CVP-Fraktion,84,82,0,84
SVP-Fraktion,77,73,0,77
Philip C. Brunner,49,49,0,49
FDP-Fraktion,38,38,0,38
Thomas Lötscher,36,36,0,36
Andreas Hausheer,34,34,0,34
Daniel Stadlin,32,32,0,32


In [62]:
df_allesgut.to_csv("KRFrames/Frame_Namen_Partei_endversion")